<br />
<div style="text-align: center;">
<font size="7">Pyro入門メモ</font>
</div>
<br />
<div style="text-align: right;">
</div>

最終更新日:2019年8月24日<br><br>
Pyroは確率的プログラミング言語であり、確率モデルを実装することができます。<br>
Pytorchで実装されている初等的な確率関数を利用でき、高速な計算やautogradが利用できます。<br>
また、transformsを用いることで好きな関数を設計することもできます。<br>

In [141]:
#pyroの環境設定は他のサイトを参考にしてください
import torch
from torch import autograd
import pyro.distributions as dist
import pyro

# 基本文法
## 確率関数
pytorchでも確率関数は用意されており、例として以下のような記法で書ける

In [142]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(0.6598)
log prob tensor(-1.1366)


一方pyroでは以下のような書き方をする。<br>
似ているが、ここで注意する相違点として、関数に名前をつける必要がある。

In [145]:
loc = 0.   # mean zero
scale = 1. # unit variance
x = pyro.sample("my_sample", pyro.distributions.Normal(loc,scale))
print(x)

tensor(-3.4848)


## モデル定義
次に簡単なモデルを実装していく
### 幾何分布
pyroにおいて、確率関数として予めベルヌーイ分布は与えられている。<br>
これを用いて幾何分布を以下のように実装できる。<br>
これは再帰的な定義になっているが、各反復の中で、それぞれ異なる名前で関数を定義する必要があるため、"x_{}"のようにして反復回数をもとにした名前を与えている<br>
他にもかなり自由度の高い実装が行える。他の応用例は後に実装する予定。

In [155]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

0


In [102]:
a = torch.distributions.Bernoulli(0.5).sample()
a.item()

0.0

In [88]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 15.0, 'sunny': 25.0}[cloudy]
    scale_temp = {'cloudy': 5.0, 'sunny': 10.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

In [101]:
weather()

('sunny', 16.511615753173828)

In [16]:
mu = torch.zeros(1)
sigma = torch.ones(1)

In [17]:
x = dist.normal(mu, sigma)

AttributeError: module 'pyro.distributions' has no attribute 'normal'

In [18]:
def model(data):
    # set prior
    alpha0 = torch.tensor(5.0)
    beta0 = torch.tensor(5.0)
    prior = dist.Beta(alpha0, beta0)

    # sample z from the prior
    z = pyro.sample("z", prior)

    with pyro.plate("data_plate", len(data)):
        # observe data
        pyro.sample("obs", dist.Bernoulli(z), obs=data)

In [19]:
d = torch.randn(10)
model(d)